WNixalo | 

---

## start

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.vision import *

In [ ]:
path = Config.data_path()/'planet'
path.mkdir(exist_ok=True)
path

## re-init all if need

In [ ]:
# ! conda install -c haasad eidl7zip -y

In [ ]:
# ! 7za -bd -y -so x {path}/train-jpg.tar.7z | tar xf - -C {path}

In [ ]:
!ls {path}

In [ ]:
# ! pip install kaggle --upgrade
# ! mkdir -p ~/.kaggle/
# ! mv ~/kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# # download
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train-jpg.tar.7z -p {path}  
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train_v2.csv -p {path}  
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f test-jpg.tar.7z -p {path}
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f test-jpg-additional.tar.7z -p {path}  
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f test_v2_file_mapping.csv -p {path}
! kaggle competitions download -c planet-understanding-the-amazon-from-space -f sample_submission_v2.csv -p {path}

In [ ]:
!ls {path}

In [ ]:
# decompress & clean
# ! unzip {path}/train_v2.csv.zip -d {path}
# ! unzip {path}/test_v2_file_mapping.csv.zip -d {path}
! unzip {path}/sample_submission_v2.csv.zip -d {path}
# ! 7za -bd -y x {path}/train-jpg.tar.7z -o{path}
# ! 7za -bd -y x {path}/test-jpg.tar.7z -o{path}
# ! 7za -bd -y x {path}/test-jpg-additional.tar.7z -o{path}
# ! tar -xf {path}/test-jpg-additional.tar -C {path}
# ! tar -xf {path}/train-jpg.tar -C {path}
# ! tar -xf {path}/test-jpg.tar -C {path}

In [ ]:
! rm -rf {path}/*.zip
# ! rm -rf {path}/*.7z
# ! rm -rf {path}/*.tar

## Data

In [ ]:
df = pd.read_csv(path/'train_v2.csv')

In [ ]:
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

In [ ]:
np.random.seed(0)

In [ ]:
src = (ImageFileList.from_folder(path)
       .label_from_csv('train_v2.csv', sep=' ', folder='train-jpg', suffix='.jpg')
       .random_split_by_pct(0.0001))

In [ ]:
data = (src.datasets(ImageMultiDataset)
        .transform(tfms, size=128)
        .databunch().normalize(imagenet_stats))

In [ ]:
data.valid_ds.ds

## MultiClass

I want to see how far I can get with a ResNet34 before going to 50.

In [ ]:
arch = models.resnet34

a threshold of 0.25 got me best results last time, so I'll see if putting that will help get more accurate metrics.

In [ ]:
f_score = partial(fbeta, thresh=0.25)
learn = create_cnn(data, arch, metrics=[accuracy_thresh, f_score])

### Stage 1 size 128

In [ ]:
lr = 0.01

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1-rn34-all')

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1-rn34-all')

### Stage 2 size 128

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn.save('stage-2-rn34-all')

In [ ]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn.save('stage-2-rn34-all')

### Stage 1 size 256

In [ ]:
data = (src.datasets(ImageMultiDataset)
        .transform(tfms, size=256)
        .databunch().normalize(imagenet_stats))

learn.data = data
data.train_ds[0][0].shape

In [ ]:
learn.freeze()

In [ ]:
lr = 1e-2/2

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1-256-rn34-all')

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1-256-rn34-all')

### Stage 2 size 256

In [ ]:
learn.unfreeze()

learn.fit_one_cycle(5, slice(1e-5, lr/5))

learn.save('stage-2-256-rn34-all')

learn.fit_one_cycle(5, slice(1e-5, lr/5))

learn.save('stage-2-256-rn34-all')

## Test set

In [ ]:
# learn.load('stage-2-256-rn34-all')

In [ ]:
idx2class = idx2class = {v:k for k,v in learn.data.train_ds.ds.class2idx.items()}

In [ ]:
!ls {path}

In [ ]:
subm_df = pd.read_csv(path/'sample_submission_v2.csv')
subdic = {c1:c2 for c1,c2 in zip(subm_df.image_name,subm_df.tags)}

In [ ]:
def set_test_datafolder(learner, test_folder):
    learner.data = (src.add_test_folder(test_folder)
                    .datasets(ImageMultiDataset)
                    .transform(tfms, size=256)
                    .databunch().normalize(imagenet_stats))

1st dataset portion

In [ ]:
set_test_datafolder(learn, 'test-jpg')
raw_preds = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
len(raw_preds), raw_preds[0].shape, raw_preds[1].shape

In [ ]:
preds = copy(raw_preds)

In [ ]:
fnames = [f.name.split('.')[0] for f in learn.data.test_ds.ds.x]

2nd dataset portion

In [ ]:
set_test_datafolder(learn, 'test-jpg-additional')
raw_preds = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
preds = (torch.cat((preds[0],raw_preds[0])), torch.cat((preds[1],raw_preds[1])))

In [ ]:
# # NOTE: this will work:
# preds = [torch.cat((preds[i],raw_preds[i])) for i in range(len(preds))]

# # but this will leave you with a <generator object <genexpr> at 0x7f91696821a8>
# preds = (torch.cat((preds[i],raw_preds[i])) for i in range(2))

In [ ]:
fnames.extend([f.name.split('.')[0] for f in learn.data.test_ds.ds.x])

## Submitting predictions to Kaggle

In [ ]:
import datetime
date = str(datetime.date.today()).replace('-','')

In [ ]:
th = 0.2775

for i,fn in enumerate(fnames):
    subdic[fn] = ' '.join(idx2class[cdx] for cdx in np.where(preds[0][i]>=th)[0])
    
subm_col = [subdic[fn] for fn in subm_df.image_name]
subm_df.tags = subm_col

In [ ]:
# !mkdir {path}/submissions

In [ ]:
sub_name = f'planet_stage-2-256-rn34-all_{date}_th{str(th)}.csv'
subm_df.to_csv(path/'submissions/'/sub_name, index=False, )
! kaggle competitions submit -c planet-understanding-the-amazon-from-space -f {path}/submissions/{sub_name} -m "fastai 1.0; 2-stg; RN34; threshold="{th}

'stage-2-256-rn34_1'|threshold|pvt score|rank|'stage-2-256-rn34'|pvt score|rank
-|-|-|-|-|-|-|-|-
#|0.5|0.89458|-|#|0.89430|-
#|0.4|0.91274|-|#|-|-
#|0.3|0.92177|-|#|0.92046|-
#|0.2|0.91901|-|#|-|-
#|0.25|0.92250|-|#|-|-
#|0.2575|0.92255|-|#|0.92201|-

Training on the entire dataset with a threshold of 0.25 gets **0.92367** pvt @ 255/983.

I may go to resnet50.

## end